## V-World를 이용해 용도지역정보를 가져올수 있는지 테스트

In [5]:
!pip install selenium webdriver_manager

In [ ]:
import os
import pandas as pd
import requests
from urllib.parse import urlencode
from pyproj import Proj, Transformer
import xml.etree.ElementTree as ET
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# EPSG:4326 (위도/경도)와 EPSG:900913 (웹 메르카토르) 프로젝션 정의
wgs84 = Proj('epsg:4326')  # WGS 84
web_mercator = Proj('epsg:900913')  # 웹 메르카토르

# Transformer 생성
transformer = Transformer.from_proj(wgs84, web_mercator)

# 위도, 경도, API 키 입력
api_key = ""

# BBOX 생성 코드
def create_bbox(lat, lng):
    delta = 0.0000001  # BBOX 크기 조정
    BBOX = f"{lng - delta:.13f}%2C{lat - delta:.13f}%2C{lng + delta:.13f}%2C{lat + delta:.13f}"
    return BBOX

# V-world API 요청을 위한 URL 생성 함수
def create_vworld_url(lat, lng, api_key):
    BBOX = create_bbox(lat, lng)
    url = (
        f"https://map.vworld.kr/proxy.do?"
        f"url=https%3A%2F%2F2d.vworld.kr%2F2DCache%2Fgis%2Fmap%2FWFS%3F"
        f"TYPENAME%3DLT_C_UQ111%26BBOX%3D{BBOX}"
        f"%26propertyname%3D(uname%2Cdyear%2Cdnum%2Csido_name%2Csigg_name%2CAG_GEOM)"
        f"%26SERVICE%3DWFS%26REQUEST%3DGetFeature"
        f"%26OUTPUTFORMAT%3Dtext%2Fxml"
        f"%26subtype%3Dgml%2F2.1.2%26VERSION%3D1.1.0%26EXCEPTIONS%3Dtext%2Fxml"
        f"%26apiKey%3D{api_key}%26srsname%3DEPSG%3A900913"
    )
    return url

# 위도와 경도를 입력받아 용도지역 정보를 요청하는 함수 (User-Agent 포함)
def get_land_use(lat, lng, api_key):
    url = create_vworld_url(lat, lng, api_key)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.content  # XML 응답 데이터 반환 (바이트로 반환)
    else:
        print(f"Error: {response.status_code}")  # 에러 코드 출력
        return None

# XML에서 용도지역 정보를 추출하는 함수
def extract_land_use(xml_content):
    if xml_content is None:
        return ["데이터를 가져오는데 실패했습니다."]

    try:
        root = ET.fromstring(xml_content.decode('utf-8'))  # UTF-8로 디코딩
    except ET.ParseError as e:
        print(f"XML 파싱 오류: {e}")
        return ["XML 파싱 오류가 발생했습니다."]

    namespaces = {
        'gml': 'http://www.opengis.net/gml',
        'sop': 'https://www.vworld.kr'
    }

    land_uses = []
    for uname in root.findall('.//sop:uname', namespaces):
        if uname is not None and uname.text:
            land_uses.append(uname.text)

    return land_uses if land_uses else ["해당 좌표에 대한 용도지역 정보를 찾을 수 없습니다."]

# 각 행에 대해 land_use 요청을 멀티스레딩으로 처리하는 함수
def process_row(row):
    latitude = row['latitude']
    longitude = row['longitude']

    # EPSG:4326에서 EPSG:900913로 변환
    x, y = transformer.transform(latitude, longitude)

    # land_use 정보를 가져오기
    land_use_info = get_land_use(x, y, api_key)
    land_uses = extract_land_use(land_use_info)

    time.sleep(1)  # 요청 간격 1초

    return {
        'address': row['address'],
        'zipcode': row['zipcode'],
        'building_name': row['building_name'],
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'land_use': ', '.join(land_uses)  # 여러 용도지역 정보가 있을 경우 쉼표로 구분
    }

# 여러 CSV 파일을 처리하고 land_use를 추가하는 함수 (멀티스레딩 적용)
def process_multiple_files(input_folder, output_folder, file_prefix, file_count):
    for i in range(63, file_count + 1): # 파일 진행상황에 따라 변경할것
        input_file = f"{input_folder}/{file_prefix}{i}.csv"
        output_file = f"{output_folder}/{file_prefix}{i}_land_use.csv"

        df = pd.read_csv(input_file)
        results = []

        # ThreadPoolExecutor를 사용하여 멀티스레딩 적용
        with ThreadPoolExecutor(max_workers=10) as executor:  # 최대 10개의 스레드 사용
            future_to_row = {executor.submit(process_row, row): row for index, row in df.iterrows()}

            for future in as_completed(future_to_row):
                result = future.result()
                results.append(result)

        # 결과를 데이터프레임으로 변환하여 CSV로 저장
        result_df = pd.DataFrame(results)
        result_df.to_csv(output_file, index=False, encoding='utf-8-sig')  # utf-8-sig로 저장
        print(f"'{output_file}'에 결과 저장 완료.")

# 폴더 및 파일 설정
input_folder = '/Users/sopung/Desktop/workspace/address/address_folder'
output_folder = '/Users/sopung/Desktop/workspace/address/address_new'
file_prefix = 'output_with_coordinates_'
file_count = 106  # 처리할 파일 수

# 지정한 경로가 존재하지 않으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 파일 처리 함수 실행
process_multiple_files(input_folder, output_folder, file_prefix, file_count)
